#Packages

In [ ]:
!pip install openai --quiet
!pip install langchain --quiet
!pip install cohere --quiet
!pip install langchain_experimental
!pip install langchain_community --quiet
!pip install langchainhub
!pip install pypdf

## API Keys

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "Your own OPEANAI_API_KEY"
os.environ['COHERE_API_KEY'] = "Your own COHERE_API_KEY"

#Better way
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")
os.environ['COHERE_API_KEY'] = userdata.get("COHERE_API_KEY")
#os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')
#os.environ["LANGCHAIN_API_KEY"]=userdata.get('LANGCHAIN_API_KEY')

# Working with LLMs

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI()
result=llm.invoke("Write a 4 line poem on AI Agents")
print(result)

In [ ]:
from langchain.llms import Cohere
llm=Cohere()
result=llm("Write a 4 line poem on AI Agents")
print(result)

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI, Cohere
from langchain.chat_models import ChatOpenAI
from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain_community.tools.tavily_search import TavilySearchResults

# LLM Limitation


In [ ]:
llm=OpenAI(temperature=0)
print(llm.invoke("Who is the current president of India?"))

In [ ]:
print(llm.invoke("Who is the IPL 2025 Winner?"))

# Agent Example

## Step-1 : Define Tools

In [ ]:
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')
tools = [TavilySearchResults(max_results=4)]

## Step-2: Define/Import the Prompt

In [ ]:
#os.environ["LANGCHAIN_API_KEY"]=userdata.get('LANGCHAIN_API_KEY')
from langchain import hub
prompt = hub.pull("hwchase17/structured-chat-agent")
print(prompt.messages[0].prompt.template)

Respond to the human as helpfully and accurately as possible. You have access to the following tools:

{tools}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}

Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation


/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


## Step-3: Define LLM and Agent

In [ ]:
llm = OpenAI(temperature=0)
agent = create_structured_chat_agent(llm,
                                     tools,
                                     prompt)

## Step-4: Agent Executor

In [ ]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

In [ ]:
agent_executor.invoke({"input": "Who is the current president of India?"})

In [ ]:
agent_executor.invoke({"input": "Who is the IPL 2025 Winner?"})

#Traditional RAG

In [ ]:
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
import os

In [ ]:
# Step 1: Load documents
!wget https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Microsoft_Earnings/TranscriptQandAFY25q4.pdf
loader = PyPDFLoader("TranscriptQandAFY25q4.pdf")
pages = loader.load()
print("Pages in the Document ==>", len(pages))
full_text =""
for page in pages:
  full_text += page.page_content

print("Pages", len(pages))
print("Lines" , len(full_text.split("\n")))
print("Words" , len(full_text.split(" ")))
print("Charecters", len(full_text))

In [ ]:
# Step 2: Split text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
chunks = text_splitter.split_documents(pages)
print(len(chunks))

In [ ]:
!pip install faiss-cpu

In [ ]:
# Step 3: Embed and store in FAISS
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(chunks, embeddings)

In [ ]:
# Step 4: Setup RetrievalQA chain
llm = OpenAI()
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    chain_type="stuff"
)

In [ ]:
#Step-5: RAG Q&A
query = "What is the LinkedIn revenue?"
response = qa_chain.invoke(query)
print("📘 Answer:\n", response["result"])

In [ ]:
# Step 5: Ask a question
query = "What is the news about copilot adoption?"
response = qa_chain.invoke(query)
print("📘 Answer:\n", response["result"])

#Agentic RAG

In [ ]:
from langchain.agents import initialize_agent, Tool

In [ ]:
# Wrap the RAG chain as a Tool
rag_tool = Tool(
    name="MS_Earnings_RAG",
    func=qa_chain.run,
    description="Useful for answering questions about MicroSoft Earnings"
)

In [ ]:
agent = initialize_agent(
    tools=[rag_tool],
    llm=llm,
    verbose=True
)

In [ ]:
query = "What is the news about copilot adoption?"
agent_response = agent.run(query)

In [ ]:
query = "Is there an impact of copilot on Microsoft Earnings?"
agent_response = agent.run(query)

## Adding one more tool

In [ ]:
!pip install yfinance

In [ ]:
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
agent = initialize_agent(
    tools=[rag_tool, YahooFinanceNewsTool()],
    llm=llm,
    verbose=True
)

In [ ]:
query = "What happened today with Microsoft stocks?"
agent_response = agent.run(query)

In [ ]:
query = "What does Microsoft’s latest financial report say? Compare them with Google"
agent_response = agent.run(query)